In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [4]:
df=pd.read_csv('data/menu_items.csv')
df.head(5)

,menu_item_id,item_name,category,price
0,101,Hamburger,American,12.95
1,102,Cheeseburger,American,13.95
2,103,Hot Dog,American,9.00
3,104,Veggie Burger,American,10.50
4,105,Mac & Cheese,American,7.00


In [5]:
## basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   menu_item_id  32 non-null     int64  
 1   item_name     32 non-null     object 
 2   category      32 non-null     object 
 3   price         32 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.1+ KB


In [6]:
df1 = pd.read_csv('data/order_details.csv')
df1.head(5)

,order_details_id,order_id,order_date,order_time,item_id
0,1,1,1/1/23,11:38:36 AM,109.0
1,2,2,1/1/23,11:57:40 AM,108.0
2,3,2,1/1/23,11:57:40 AM,124.0
3,4,2,1/1/23,11:57:40 AM,117.0
4,5,2,1/1/23,11:57:40 AM,129.0


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12234 entries, 0 to 12233
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_details_id  12234 non-null  int64  
 1   order_id          12234 non-null  int64  
 2   order_date        12234 non-null  object 
 3   order_time        12234 non-null  object 
 4   item_id           12097 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 478.0+ KB


In [ ]:
from sqlalchemy import create_engine, inspect, text

## connect to MSSQL database using Windows Authentication
server = "DESKTOP-G3STKJF\\SQLEXPRESS"  # Server instance name
database = "restaurant_order"

# Connection string for Windows Authentication
connection_string = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"

# First, connect to master database to check if database exists and create if needed
master_connection_string = f"mssql+pyodbc://{server}/master?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"

try:
    # Connect to master database
    engine_master = create_engine(master_connection_string)
    
    # Check if database exists
    inspector = inspect(engine_master)
    databases = inspector.get_schema_names()
    
    if database not in databases:
        print(f"Database '{database}' does not exist. Creating...")
        with engine_master.connect() as conn:
            conn.execute(text(f"CREATE DATABASE [{database}]"))
            conn.commit()
        print(f"Database '{database}' created successfully!")
    else:
        print(f"Database '{database}' already exists.")
    
    engine_master.dispose()
    
    # Now connect to the target database and load data
    engine = create_engine(connection_string)
    
    ## load the data to MSSQL database
    table_name = "menu_items"
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Data loaded into '{table_name}' table")
    
    table_name_2 = "order_details"
    df1.to_sql(table_name_2, engine, if_exists='replace', index=False)
    print(f"Data loaded into '{table_name_2}' table")
    
    print(f"\n✓ All data successfully loaded into '{database}' database!")
    
except Exception as e:
    print(f"Error: {e}")
    print(f"\nTroubleshooting tips:")
    print(f"1. Ensure SQL Server is running")
    print(f"2. Verify server name: {server}")
    print(f"3. Check that ODBC Driver 17 is installed")
    print(f"4. Verify Windows Authentication is enabled")

InterfaceError: (pyodbc.InterfaceError) ('28000', '[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user \'DESKTOP-G3STKJF\\ENG.YAHYA\'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open database "restaurant_order" requested by the login. The login failed. (4060); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user \'DESKTOP-G3STKJF\\ENG.YAHYA\'. (18456); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open database "restaurant_order" requested by the login. The login failed. (4060)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)